In [193]:
import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

BATCH_SIZE=10
LR=0.01
EPOCH=1
PREDICT_TIME=15

## Dataset

In [185]:
class LOLDataset(Dataset):
    def __init__(self,moneyData,label):
        self.money=[]
        for i in range(PREDICT_TIME):
            self.money.append([line[i] for line in moneyData])
        self.label=[i for i in label]
    
    def __getitem__(self,item):
        # print(item,"###")
        return torch.tensor([[torch.scalar_tensor(x[item])] for x in self.money]),torch.tensor(self.label[item])
    
    def __len__(self):
        return len(self.label)
            

In [186]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df["golddiff"]
# df["bResult"]
gold = [line[1:-1].split(",") for line in df["golddiff"]]
gold=[[int(x) for x in line]for line in gold]

In [187]:
dataset=LOLDataset(gold,df["bResult"])
train_size=int(0.8*len(dataset))
test_size = len(dataset)-train_size

trainDataset,testDataset=random_split(dataset,[train_size,test_size])

trainLoader=DataLoader(trainDataset,batch_size=32,shuffle=True)
testLoader=DataLoader(testDataset,batch_size=32)

## RNN Network

In [188]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()

        self.rnn=nn.LSTM(
            input_size=1,
            hidden_size=128,
            num_layers=1,
            batch_first=True
        )

        self.out = nn.Linear(128, 2)
    def forward(self,x):
        r_out,(h_n,h_c)=self.rnn(x,None)
        out = self.out(r_out[:, -1, :])
        return out

In [189]:
model=RNN()
print(model)

RNN(
  (rnn): LSTM(1, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=2, bias=True)
)


## Train

In [195]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (x, y) in tqdm(enumerate(trainLoader)):
        b_x = Variable(x)
        b_y = Variable(y)
        
        output = model(b_x)
        loss = loss_func(output, b_y)
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

191it [03:52,  1.22s/it]


## Test

In [207]:
model.eval()
sum = torch.tensor(0)
count=0
for step,(x,y) in tqdm(enumerate(testLoader)):
    b_x = Variable(x)
    b_y = Variable(y)
    predict=model(b_x)
    predict=torch.max(predict,1).indices
    sum+=torch.sum(torch.abs(b_y-predict))
    count+=32
print(1-sum/count)

48it [00:42,  1.12it/s]

tensor(0.7135)
